In [1]:
import mcts
import keras
from game import Board
from tqdm import tqdm
import numpy as np
import pdb
import keras

In [2]:
model = keras.models.load_model('../saved_models/supervised_model_tablut_md.keras')

In [3]:
def fst(x):
        return x[0]

positions_data = []
move_probs_data = []
values_data = []
unmasked_probs = []

g = Board()
g.set_starting_position()

while (not fst(g.is_terminal())):
    positions_data.append(g.to_network_input_multidim())

    root_edge = mcts.Edge(None, None)
    root_edge.N = 1
    root_node = mcts.Node(g, root_edge)
    mcts_searcher = mcts.MCTS(model)

    move_probs = mcts_searcher.search(root_node)
    unmasked_probs.append(move_probs)
    output_vec = [0.0 for x in range(0, Board.LEN_OUTPUT_INDEX)]
    for (move, prob, _, _) in move_probs:
        move_idx = g.get_network_output_index(move)
        output_vec[move_idx] = prob

    rand_idx = np.random.multinomial(1, output_vec)
    idx = np.where(rand_idx==1)[0][0]
    next_move = None

    for move, _, _, _ in move_probs:
        move_idx = g.get_network_output_index(move)
        if(move_idx == idx):
            next_move = move
    if g.turn == Board.WHITE:
        values_data.append(1)
    else:
        values_data.append(-1)
    move_probs_data.append(output_vec)
    g.apply_move(next_move)
else:
    _, winner = g.is_terminal()
    for i in range(0, len(move_probs_data)):
        if winner == Board.BLACK:
            values_data[i] = values_data[i] * -1.0
        if winner == Board.WHITE:
            values_data[i] = values_data[i] * 1.0
        if winner == Board.EMPTY:
            values_data[i] = values_data[i] * 0.0

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━

In [4]:
values_data

[1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 1.0]

In [5]:
move_probs_data

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,

In [6]:
unmasked_probs

[[((22, 23), 0.0, 0, 0),
  ((22, 24), 0.0, 0, 0),
  ((22, 25), 0.0, 0, 0),
  ((22, 26), 0.0, 0, 0),
  ((22, 21), 0.0, 0, 0),
  ((22, 20), 0.0, 0, 0),
  ((22, 19), 0.0, 0, 0),
  ((22, 18), 0.0, 0, 0),
  ((31, 32), 0.1, 10, -0.03390525914728641),
  ((31, 33), 0.0, 0, 0),
  ((31, 34), 0.0, 0, 0),
  ((31, 30), 0.02, 2, -0.19799486576812342),
  ((31, 29), 0.0, 0, 0),
  ((31, 28), 0.0, 0, 0),
  ((38, 47), 0.0, 0, 0),
  ((38, 56), 0.0, 0, 0),
  ((38, 65), 0.0, 0, 0),
  ((38, 74), 0.0, 0, 0),
  ((38, 29), 0.0, 0, 0),
  ((38, 20), 0.0, 0, 0),
  ((38, 11), 0.0, 0, 0),
  ((38, 2), 0.0, 0, 0),
  ((39, 48), 0.12, 12, 0.028735329397022724),
  ((39, 57), 0.0, 0, 0),
  ((39, 66), 0.0, 0, 0),
  ((39, 30), 0.1, 10, 0.0327584398444742),
  ((39, 21), 0.0, 0, 0),
  ((39, 12), 0.0, 0, 0),
  ((41, 50), 0.16, 16, 0.07053342173458077),
  ((41, 59), 0.0, 0, 0),
  ((41, 68), 0.0, 0, 0),
  ((41, 32), 0.1, 10, 0.0249212296679616),
  ((41, 23), 0.0, 0, 0),
  ((41, 14), 0.0, 0, 0),
  ((42, 51), 0.0, 0, 0),
  ((42, 6

In [ ]:
model = keras.models.load_model("../saved_models/supervised_model_tablut_md.keras")
mcts_searcher = mcts.MCTS(model)
learner = ReinfLearn(model)

for i in range(0, 3):
    print(f"Training Iteration: {i}")
    all_pos = []
    all_move_probs = []
    all_values = []
    for j in tqdm(range(0, 3)):
        pos, move_probs, values = learner.play_game()
        all_pos += pos
        all_move_probs += move_probs
        all_values += values

        print(pos[1])
        print(move_probs[1])

    np_pos = np.array(all_pos)
    np_probs = np.array(all_move_probs)
    np_vals = np.array(all_values)
    print(f'positions: {len(np_pos)} | probs: {len(np_probs)} | vals: {len(np_vals)}')
    model.fit(np_pos, [np_probs, np_vals], epochs = 256, batch_size=16, verbose=0)
    if i == 10:
        model.save(f'model_it{i}.keras')